In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import pickle
import numpy as np

nltk.download('punkt')

# Example dataset
data = [
    {"intent": "greeting", "patterns": ["Hi", "Hello", "Hey"], "responses": ["Hello!", "Hi there!"]},
    {"intent": "goodbye", "patterns": ["Bye", "See you later", "Goodbye"], "responses": ["Goodbye!", "See you later!"]}
]

# Data preprocessing
lemmatizer = WordNetLemmatizer()

words = []
classes = []
documents = []
responses = {}

for intent in data:
    for pattern in intent["patterns"]:
        # Tokenize and lemmatize words
        tokens = word_tokenize(pattern)
        words.extend(tokens)
        documents.append((tokens, intent["intent"]))
        classes.append(intent["intent"])

    responses[intent["intent"]] = intent["responses"]

words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in "?"]
words = sorted(list(set(words)))

# Create training data
training = []
output_empty = [0] * len(classes)

for document in documents:
    bag = []
    pattern_words = document[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

    for word in words:
        bag.append(1) if word in pattern_words else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

X_train = np.array([i[0] for i in training])
y_train = np.array([i[1] for i in training])

# Train a model (example using a simple neural network)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

model = Sequential([
    Dense(128, input_shape=(len(X_train[0]),), activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(len(y_train[0]), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, batch_size=5, verbose=1)

# Save the model and preprocessing objects
model.save("chatbot_model.h5")
pickle.dump({"words": words, "classes": classes, "responses": responses}, open("chatbot_data.pkl", "wb"))
